In [10]:
import numpy as np
import pandas as pd
import requests, re
from bs4 import BeautifulSoup
from bs4 import Tag, NavigableString, Comment


# Create a function to parse through web pages

def parse_page(_d):
    data = filter(lambda x:x != '\n', [i for i in _d.find('table', {'id':'Browse'}).td.contents if isinstance(i, str) or i.name == 'a'])
    _next = next(data, None)
    while _next is not None:
        new_d, _n = {'title':_next.text, 'link':_next['href']}, next(data, None)
        if hasattr(_n, 'text'):
            yield new_d
            yield {'title':_n.text, 'link':_n['href'], 'about':next(data)}
        else:
            yield {**new_d, 'about':_n}   
        _next = next(data, None)  
        

# Identify the web page to start at and create a parsed text list with parse_page

d, r = BeautifulSoup(requests.get('https://www.mtsamples.com/site/pages/browse.asp?type=24-Gastroenterology&page=1').text, 'html.parser'), []
r.append(list(parse_page(d)))
_c = [i for i in d.find('div', {'class':'Contrast'}).find_all('a') if i.text == '>']


# Use a while loop to continuously iterate over the links as long as the "next" (>) button is present
# At each iteration, scrape the next page and extract the headers with the link and description

while _c:
    d = BeautifulSoup(requests.get(f'https://www.mtsamples.com{_c[0]["href"]}').text, 'html.parser')
    r.append(list(parse_page(d)))
    _c = [i for i in d.find('div', {'class':'Contrast'}).find_all('a') if i.text == '>']
                          

In [11]:
# Specify medical specialty of web pages

gastroenterology_web_pages = r
len(gastroenterology_web_pages)

23

In [12]:
gastroenterology_web_pages

[[{'title': 'Abdominal Abscess I&D',
   'link': '/site/pages/sample.asp?Type=24-Gastroenterology&Sample=2332-Abdominal Abscess I&D',
   'about': 'Incision and drainage (I&D) of abdominal abscess, excisional debridement of nonviable and viable skin, subcutaneous tissue and muscle, then removal of foreign body.'},
  {'title': 'Abdominal Exploration',
   'link': '/site/pages/sample.asp?Type=24-Gastroenterology&Sample=2038-Abdominal Exploration',
   'about': 'Congenital chylous ascites and chylothorax and rule out infradiaphragmatic lymphatic leak.   Diffuse intestinal and mesenteric lymphangiectasia.  '},
  {'title': 'Abdominal Pain - Consult',
   'link': '/site/pages/sample.asp?Type=24-Gastroenterology&Sample=1921-Abdominal Pain - Consult',
   'about': 'The patient presented to the emergency room last evening with approximately 7- to 8-day history of abdominal pain which has been persistent.'},
  {'title': 'Abscess Excision',
   'link': '/site/pages/sample.asp?Type=24-Gastroenterology&Sa

In [13]:
# Create a url list for the medical records within the specified medical specialty 

gastroenterology_url_list = []

for i in gastroenterology_web_pages:
    for j in i:
        gastroenterology_url_list.append('https://www.mtsamples.com' + j['link'])
        
len(gastroenterology_url_list)

230

In [14]:
print(gastroenterology_url_list)

['https://www.mtsamples.com/site/pages/sample.asp?Type=24-Gastroenterology&Sample=2332-Abdominal Abscess I&D', 'https://www.mtsamples.com/site/pages/sample.asp?Type=24-Gastroenterology&Sample=2038-Abdominal Exploration', 'https://www.mtsamples.com/site/pages/sample.asp?Type=24-Gastroenterology&Sample=1921-Abdominal Pain - Consult', 'https://www.mtsamples.com/site/pages/sample.asp?Type=24-Gastroenterology&Sample=2330-Abscess Excision', 'https://www.mtsamples.com/site/pages/sample.asp?Type=24-Gastroenterology&Sample=1152-Admission History & Physical - Nausea', 'https://www.mtsamples.com/site/pages/sample.asp?Type=24-Gastroenterology&Sample=2464-Adrenalectomy & Umbilical Hernia Repair', 'https://www.mtsamples.com/site/pages/sample.asp?Type=24-Gastroenterology&Sample=2166-Air Under Diaphragm - Consult', 'https://www.mtsamples.com/site/pages/sample.asp?Type=24-Gastroenterology&Sample=1975-Appendectomy', 'https://www.mtsamples.com/site/pages/sample.asp?Type=24-Gastroenterology&Sample=2298-Ap

In [15]:
def get_medical_notes(url_link):

    global medical_notes
    medical_notes = []
    
    
    # We will loop through each of the url links provided as input for the function

    for i in url_link:

        url = i
        res = requests.get(url)
        res.raise_for_status()
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
        
        
        # Here we can get page_title from <h1>

        title_el = soup.find('h1')
        page_title = title_el.text.strip()
        first_hr = title_el.find_next_sibling('hr')
        
        
        # Here we can get the page_title description after we see the text 'description'

        description_title = title_el.find_next_sibling('b', text=re.compile('description', flags=re.I))
        description_text_parts = []
        for s in description_title.next_siblings:
            if s is first_hr:
                break
            if isinstance(s, Tag):
                description_text_parts.append(s.text.strip())
            elif isinstance(s, NavigableString):
                description_text_parts.append(str(s).strip())
        description_text = '\n'.join(p for p in description_text_parts if p.strip())

        
        # Titles are all bold and uppercase
        
        titles = [b for b in first_hr.find_next_siblings('b') if b.text.strip().isupper()]
        
        
        # We can find the text between the titles and assign it to the title we see earlier

        docs = []
        
        for t in titles:
            text_parts = []
            for s in t.next_siblings:
                
                # go until next title
                
                if s in titles:
                    break
                if isinstance(s, Comment):
                    continue
                if isinstance(s, Tag):
                    if s.name == 'div':
                        break
                    text_parts.append(s.text.strip())
                elif isinstance(s, NavigableString):
                    text_parts.append(str(s).strip())
                    
            text = '\n'.join(p for p in text_parts if p.strip())
            
            docs.append({
                'title': t.text.strip(),
                'text': text
            })
            
            
        # Clean the text in docs such that we have one full_text document of medical notes
        
        lis = []

        for j in docs:
            for k in j.values():
                lis.append(k.rstrip('\n'))

        full_text = ''.join(lis)
        
        
        # Append to our medical_notes list

        medical_notes.append(full_text)

    len(medical_notes)

In [16]:
# Get and clean all medical notes for the specified medical specialty 

get_medical_notes(gastroenterology_url_list)

gastroenterology_medical_notes = medical_notes
len(gastroenterology_medical_notes)

230

In [17]:
gastroenterology_medical_notes

['PREOPERATIVE DIAGNOSIS:Abdominal wall abscess.POSTOPERATIVE DIAGNOSIS:Abdominal wall abscess.PROCEDURE:Incision and drainage (I&D) of abdominal abscess, excisional debridement of nonviable and viable skin, subcutaneous tissue and muscle, then removal of foreign body.ANESTHESIA:LMA.INDICATIONS:Patient is a pleasant 60-year-old gentleman, who initially had a sigmoid colectomy for diverticular abscess, subsequently had a dehiscence with evisceration.  Came in approximately 36 hours ago with pain across his lower abdomen.  CT scan demonstrated presence of an abscess beneath the incision.  I recommended to the patient he undergo the above-named procedure.  Procedure, purpose, risks, expected benefits, potential complications, alternatives forms of therapy were discussed with him, and he was agreeable to surgery.FINDINGS:The patient was found to have an abscess that went down to the level of the fascia.  The anterior layer of the fascia was fibrinous and some portions necrotic.  This was e

In [21]:
# Return medical notes and the labeled medical specialty as a dataframe

gastroenterology_df = pd.DataFrame({'Medical Specialty': 'Gastroenterology', 'Medical Notes': gastroenterology_medical_notes})
print(len(gastroenterology_df))
gastroenterology_df.head()


230


,Medical Specialty,Medical Notes
0,Gastroenterology,PREOPERATIVE DIAGNOSIS:Abdominal wall abscess....
1,Gastroenterology,PREOPERATIVE DIAGNOSES:1. Congenital chylous ...
2,Gastroenterology,CHIEF COMPLAINT:Abdominal pain.HISTORY OF PRES...
3,Gastroenterology,PREOPERATIVE DIAGNOSIS:Recurrent re-infected s...
4,Gastroenterology,CHIEF COMPLAINT:Nausea.PRESENT ILLNESS:The pat...


In [19]:
import pickle

# pickle dataframe as a checkpoint

pickle_out = open("gastroenterology_df.pickle", "wb") # write file to bytes
pickle.dump(gastroenterology_df, pickle_out)
pickle_out.close()